In [1]:
import pandas as pd
import numpy as np
import joblib
import warnings
from IPython.display import display

warnings.filterwarnings('ignore')

print("=======================================================")
print("   🤖 QUANT ENGINE v19 (MARKET AWARE + DEADLOCK)       ")
print("=======================================================")

# ==============================================================================
# 1. CONFIGURATION
# ==============================================================================
BANKROLL = 30.00        
MIN_EDGE = 0.01           
MAX_ODDS_CAP = 6.00       
MAX_STAKE_PCT = 0.12      
KELLY_FRACTION = 0.25     

# --- FIXTURES (Odds are now INPUT FEATURES too!) ---
fixtures = [
    {"Home": "Manchester United", "Away": "Newcastle United", "Odds_1": 2.53, "Odds_X": 3.61, "Odds_2": 2.68},
    {"Home": "Nottingham Forest", "Away": "Manchester City", "Odds_1": 5.29, "Odds_X": 4.45, "Odds_2": 1.58},
    {"Home": "Arsenal",           "Away": "Brighton & Hove Albion", "Odds_1": 1.40, "Odds_X": 4.92, "Odds_2": 7.85},
    {"Home": "Brentford",         "Away": "Bournemouth",      "Odds_1": 2.29, "Odds_X": 3.60, "Odds_2": 3.03},
    {"Home": "Burnley",           "Away": "Everton",          "Odds_1": 4.01, "Odds_X": 3.35, "Odds_2": 2.01},
    {"Home": "Liverpool",         "Away": "Wolverhampton Wanderers", "Odds_1": 1.24, "Odds_X": 6.59, "Odds_2": 11.28},
    {"Home": "West Ham United",   "Away": "Fulham",           "Odds_1": 2.68, "Odds_X": 3.41, "Odds_2": 2.63},
    {"Home": "Chelsea",           "Away": "Aston Villa",      "Odds_1": 1.85, "Odds_X": 3.88, "Odds_2": 4.03},
    {"Home": "Sunderland",        "Away": "Leeds United",     "Odds_1": 2.58, "Odds_X": 3.23, "Odds_2": 2.86},
    {"Home": "Crystal Palace",    "Away": "Tottenham Hotspur", "Odds_1": 2.29, "Odds_X": 3.30, "Odds_2": 3.25},
]

# ==============================================================================
# 2. LOAD MODEL & DATA
# ==============================================================================
try:
    artifacts = joblib.load('football_model_final.pkl')
    model = artifacts['model']
    features = artifacts['features']
    current_elos = artifacts['elo_dict']
    df_recent = artifacts['df_recent']
    print(f"✅ Model Loaded. Features expected: {len(features)}")
    
    # Check if 'market_prob_home' is in features (Validation)
    if 'market_prob_home' in features:
        print("   -> Market Features detected (Good).")
    else:
        print("   ⚠️ WARNING: Model does not seem to be Market Aware.")
        
except FileNotFoundError:
    print("❌ Error: 'football_model_final.pkl' not found.")
    exit()

def get_stats(team):
    # Get last known stats
    rows = df_recent[(df_recent['home_team_name'] == team) | (df_recent['away_team_name'] == team)]
    if len(rows) == 0: return None
    last = rows.sort_values('date').iloc[-1]
    
    prefix = 'home_' if last['home_team_name'] == team else 'away_'
    
    stats = {}
    # Dynamic extraction of rolling features
    valid_cols = [c for c in df_recent.columns if 'roll_' in c]
    for col in valid_cols:
        if col.startswith(prefix):
            generic_name = col.replace(prefix, '')
            stats[generic_name] = last[col]
            
    return stats

# ==============================================================================
# 3. ANALYSIS LOOP
# ==============================================================================
opportunities = []
ELITE_TEAMS = ["Manchester City", "Liverpool", "Arsenal", "Real Madrid", "Barcelona", "Inter", "Bayern Munich"]

print(f"\n🔍 ANALYZING {len(fixtures)} MATCHES...")

for f in fixtures:
    h_team, a_team = f['Home'], f['Away']
    h_stats, a_stats = get_stats(h_team), get_stats(a_team)
    h_elo, a_elo = current_elos.get(h_team, 1500), current_elos.get(a_team, 1500)
    
    if not h_stats or not a_stats: continue

    # --- FEATURE CONSTRUCTION ---
    input_data = {}
    
    # 1. Base Stats (ELO, Rest)
    input_data['diff_elo'] = (h_elo + 70) - a_elo 
    input_data['home_elo'] = h_elo
    input_data['away_elo'] = a_elo
    input_data['diff_rest'] = 0
    
    # 2. MARKET ODDS FEATURES (NEW!)
    # Calculate implied probabilities from the odds provided in the fixture list
    imp_h = 1 / f['Odds_1']
    imp_d = 1 / f['Odds_X']
    imp_a = 1 / f['Odds_2']
    
    # Normalize (remove bookie margin)
    total_imp = imp_h + imp_d + imp_a
    input_data['market_prob_home'] = imp_h / total_imp
    input_data['market_prob_draw'] = imp_d / total_imp
    input_data['market_prob_away'] = imp_a / total_imp
    input_data['has_odds'] = 1
    
    # 3. Dynamic Rolling & Diffs
    for feat in features:
        if feat in input_data: continue
        
        if 'home_roll_' in feat:
            key = feat.replace('home_', '')
            input_data[feat] = h_stats.get(key, 0)
        elif 'away_roll_' in feat:
            key = feat.replace('away_', '')
            input_data[feat] = a_stats.get(key, 0)
        elif 'diff_' in feat:
            key = feat.replace('diff_', 'roll_')
            input_data[feat] = h_stats.get(key, 0) - a_stats.get(key, 0)

    # 4. Predict
    input_df = pd.DataFrame([input_data]).reindex(columns=features, fill_value=0)
    probs = model.predict_proba(input_df)[0]
    p_away, p_draw, p_home = probs[0], probs[1], probs[2]

    # --- DEADLOCK & GOLIATH LOGIC ---
    
    # Deadlock Shift (Still valid for Market Models)
    # If the Model AND the Market agree it's tight, Draw is very likely
    gap = abs(p_home - p_away)
    is_tight = gap < 0.12
    
    if is_tight:
        shift = 0.08
        p_draw += shift
        p_home -= (shift / 2)
        p_away -= (shift / 2)
        
    # Goliath Safety
    block_underdog = False
    if (a_team in ELITE_TEAMS and h_elo < a_elo - 200) or (h_team in ELITE_TEAMS and a_elo < h_elo - 200):
        block_underdog = True

    print(f"{h_team[:10]} vs {a_team[:10]}: H {p_home:.2f} | D {p_draw:.2f} | A {p_away:.2f} {'[Tight]' if is_tight else ''}")

    # --- VALUE FINDER ---
    bets_for_match = []
    
    for outcome, p, odds, label in [
        ('HOME', p_home, f['Odds_1'], h_team),
        ('DRAW', p_draw, f['Odds_X'], 'Draw'),
        ('AWAY', p_away, f['Odds_2'], a_team)
    ]:
        if odds > MAX_ODDS_CAP: continue
        
        if block_underdog:
            # Only bet favorite in Goliath matches
            is_fav = (p == max(p_home, p_draw, p_away))
            if not is_fav: continue
            
        edge = (p * odds) - 1
        min_p = 0.23 if outcome == 'DRAW' else 0.20
        
        if edge > MIN_EDGE and p > min_p:
            bets_for_match.append({'Type': outcome, 'Team': label, 'Odds': odds, 'Prob': p, 'Edge': edge})

    # --- SELECTION ---
    if bets_for_match:
        best = sorted(bets_for_match, key=lambda x: x['Edge'], reverse=True)[0]
        
        # Deadlock Override
        if is_tight:
            draw_bet = next((b for b in bets_for_match if b['Type'] == 'DRAW'), None)
            if draw_bet:
                # Prefer Draw unless winner edge is significantly better
                if best['Edge'] < (draw_bet['Edge'] + 0.15):
                    best = draw_bet
                    best['Reason'] = "Deadlock Value"
        
        best['Match'] = f"{h_team} vs {a_team}"
        opportunities.append(best)

# ==============================================================================
# 4. PORTFOLIO
# ==============================================================================
if opportunities:
    res = pd.DataFrame(opportunities)
    
    def calculate_stake(row):
        b = row['Odds'] - 1
        p = row['Prob']
        q = 1 - p
        f = ((b * p) - q) / b
        return max(0, min(f * KELLY_FRACTION, MAX_STAKE_PCT))

    res['Stake_Pct'] = res.apply(calculate_stake, axis=1)
    res['Stake'] = res['Stake_Pct'] * BANKROLL
    res['Profit'] = res['Stake'] * (res['Odds'] - 1)
    
    res = res[res['Stake'] > 0.01].sort_values('Stake', ascending=False)
    
    display_df = res[['Match', 'Type', 'Team', 'Odds', 'Prob', 'Edge', 'Stake', 'Profit']].copy()
    display_df.columns = ['Match', 'Bet Type', 'Selection', 'Odds', 'Win %', 'Edge', 'Stake ($)', 'Profit ($)']
    
    styled = (display_df.style
        .format({'Odds': '{:.2f}', 'Win %': '{:.1%}', 'Edge': '{:.1%}', 'Stake ($)': '${:.2f}', 'Profit ($)': '${:.2f}'})
        .background_gradient(subset=['Win %'], cmap='Greens', vmin=0.2, vmax=0.8)
        .background_gradient(subset=['Edge'], cmap='Blues', vmin=0.0, vmax=0.5)
        .bar(subset=['Stake ($)'], color='#ff6b6b', vmin=0)
        .bar(subset=['Profit ($)'], color='#51cf66', vmin=0)
        .set_caption(f"🚀 FINAL MARKET-AWARE SLIP | Bankroll: ${BANKROLL}")
    )
    display(styled)
    
    print(f"\n💰 Total Risk: ${res['Stake'].sum():.2f}")
else:
    print("\n📉 No Value Bets Found.")

   🤖 QUANT ENGINE v19 (MARKET AWARE + DEADLOCK)       
✅ Model Loaded. Features expected: 60
   -> Market Features detected (Good).

🔍 ANALYZING 10 MATCHES...
Manchester vs Newcastle : H 0.33 | D 0.47 | A 0.20 
Nottingham vs Manchester: H 0.22 | D 0.14 | A 0.64 
Arsenal vs Brighton &: H 0.85 | D 0.02 | A 0.12 
Brentford vs Bournemout: H 0.34 | D 0.33 | A 0.33 [Tight]
Burnley vs Everton: H 0.16 | D 0.03 | A 0.81 
Liverpool vs Wolverhamp: H 0.91 | D 0.01 | A 0.08 
West Ham U vs Fulham: H 0.16 | D 0.16 | A 0.67 
Chelsea vs Aston Vill: H 0.76 | D 0.02 | A 0.22 
Sunderland vs Leeds Unit: H 0.34 | D 0.30 | A 0.36 [Tight]
Crystal Pa vs Tottenham : H 0.60 | D 0.20 | A 0.20 


,Match,Bet Type,Selection,Odds,Win %,Edge,Stake ($),Profit ($)
2,Arsenal vs Brighton & Hove Albion,HOME,Arsenal,1.40,85.3%,19.4%,$3.60,$1.44
4,Burnley vs Everton,AWAY,Everton,2.01,81.4%,63.6%,$3.60,$3.64
5,Liverpool vs Wolverhampton Wanderers,HOME,Liverpool,1.24,90.6%,12.4%,$3.60,$0.86
7,Chelsea vs Aston Villa,HOME,Chelsea,1.85,75.8%,40.3%,$3.55,$3.02
6,West Ham United vs Fulham,AWAY,Fulham,2.63,67.1%,76.6%,$3.52,$5.75
9,Crystal Palace vs Tottenham Hotspur,HOME,Crystal Palace,2.29,60.1%,37.6%,$2.19,$2.82
0,Manchester United vs Newcastle United,DRAW,Draw,3.61,46.8%,68.9%,$1.98,$5.17
3,Brentford vs Bournemouth,DRAW,Draw,3.60,32.5%,17.1%,$0.49,$1.28
1,Nottingham Forest vs Manchester City,HOME,Nottingham Forest,5.29,22.2%,17.6%,$0.31,$1.32
8,Sunderland vs Leeds United,AWAY,Leeds United,2.86,36.2%,3.4%,$0.14,$0.26



💰 Total Risk: $22.98


In [ ]:
import pandas as pd
import numpy as np
import joblib
import warnings
from IPython.display import display

warnings.filterwarnings('ignore')

print("=======================================================")
print("   🤖 QUANT ENGINE v19 (MARKET AWARE + PLAYER STATS)   ")
print("=======================================================")

# ==============================================================================
# 1. CONFIGURATION
# ==============================================================================
BANKROLL = 30.00        
MIN_EDGE = 0.01           # 1% Edge (Model is market-aware, so small edges are real)
MAX_ODDS_CAP = 6.00       
MAX_STAKE_PCT = 0.12      
KELLY_FRACTION = 0.25     

# --- FIXTURES (Boxing Day) ---
fixtures = [
    {"Home": "Manchester United", "Away": "Newcastle United", "Odds_1": 2.53, "Odds_X": 3.61, "Odds_2": 2.68},
    {"Home": "Nottingham Forest", "Away": "Manchester City", "Odds_1": 5.29, "Odds_X": 4.45, "Odds_2": 1.58},
    {"Home": "Arsenal",           "Away": "Brighton & Hove Albion", "Odds_1": 1.40, "Odds_X": 4.92, "Odds_2": 7.85},
    {"Home": "Brentford",         "Away": "Bournemouth",      "Odds_1": 2.29, "Odds_X": 3.60, "Odds_2": 3.03},
    {"Home": "Burnley",           "Away": "Everton",          "Odds_1": 4.01, "Odds_X": 3.35, "Odds_2": 2.01},
    {"Home": "Liverpool",         "Away": "Wolverhampton Wanderers", "Odds_1": 1.24, "Odds_X": 6.59, "Odds_2": 11.28},
    {"Home": "West Ham United",   "Away": "Fulham",           "Odds_1": 2.68, "Odds_X": 3.41, "Odds_2": 2.63},
    {"Home": "Chelsea",           "Away": "Aston Villa",      "Odds_1": 1.85, "Odds_X": 3.88, "Odds_2": 4.03},
    {"Home": "Sunderland",        "Away": "Leeds United",     "Odds_1": 2.58, "Odds_X": 3.23, "Odds_2": 2.86},
    {"Home": "Crystal Palace",    "Away": "Tottenham Hotspur", "Odds_1": 2.29, "Odds_X": 3.30, "Odds_2": 3.25},
]

# ==============================================================================
# 2. LOAD MODEL & DATA
# ==============================================================================
try:
    artifacts = joblib.load('football_model_final.pkl')
    model = artifacts['model']
    features = artifacts['features']
    current_elos = artifacts['elo_dict']
    df_recent = artifacts['df_recent']
    print(f"✅ Model Loaded. Features: {len(features)}")
except FileNotFoundError:
    print("❌ Error: 'football_model_final.pkl' not found.")
    exit()

def get_stats(team):
    # Get last known stats for a team
    rows = df_recent[(df_recent['home_team_name'] == team) | (df_recent['away_team_name'] == team)]
    if len(rows) == 0: return None
    last = rows.sort_values('date').iloc[-1]
    
    prefix = 'home_' if last['home_team_name'] == team else 'away_'
    
    # DYNAMIC EXTRACTION
    stats = {}
    valid_cols = [c for c in df_recent.columns if 'roll_' in c]
    
    for col in valid_cols:
        if col.startswith(prefix):
            generic_name = col.replace(prefix, '')
            stats[generic_name] = last[col]
            
    return stats

# ==============================================================================
# 3. ANALYSIS LOOP
# ==============================================================================
opportunities = []
ELITE_TEAMS = ["Manchester City", "Liverpool", "Arsenal"]

print(f"\n🔍 ANALYZING {len(fixtures)} MATCHES...")

for f in fixtures:
    h_team, a_team = f['Home'], f['Away']
    h_stats, a_stats = get_stats(h_team), get_stats(a_team)
    h_elo, a_elo = current_elos.get(h_team, 1500), current_elos.get(a_team, 1500)
    
    if not h_stats or not a_stats: continue

    # --- INPUT CONSTRUCTION ---
    input_data = {}
    
    # 1. Market Probabilities (Calculated exactly like training)
    imp_h = 1 / f['Odds_1']
    imp_d = 1 / f['Odds_X']
    imp_a = 1 / f['Odds_2']
    
    # Normalize (Remove Bookie Vig) to get True Market Probability
    market_sum = imp_h + imp_d + imp_a
    input_data['market_prob_home'] = imp_h / market_sum
    input_data['market_prob_draw'] = imp_d / market_sum
    input_data['market_prob_away'] = imp_a / market_sum
    input_data['has_odds'] = 1
    
    # 2. Base Features
    input_data['diff_elo'] = (h_elo + 70) - a_elo
    input_data['home_elo'] = h_elo
    input_data['away_elo'] = a_elo
    input_data['diff_rest'] = 0
    
    # 3. Dynamic Features (Stats)
    for feat in features:
        if feat in input_data: continue
        
        # Handle Rolling
        if 'home_roll_' in feat:
            key = feat.replace('home_', '')
            input_data[feat] = h_stats.get(key, 0)
        elif 'away_roll_' in feat:
            key = feat.replace('away_', '')
            input_data[feat] = a_stats.get(key, 0)
        # Handle Differentials
        elif 'diff_' in feat:
            key = feat.replace('diff_', 'roll_') # Reconstruct key (e.g. roll_players_xA)
            # The feature list has 'diff_players_xA', stats has 'roll_players_xA'
            input_data[feat] = h_stats.get(key, 0) - a_stats.get(key, 0)

    # Predict
    input_df = pd.DataFrame([input_data]).reindex(columns=features, fill_value=0)
    probs = model.predict_proba(input_df)[0]
    p_away, p_draw, p_home = probs[0], probs[1], probs[2]

    print(f"{h_team[:10]} vs {a_team[:10]}: H {p_home:.2f} | D {p_draw:.2f} | A {p_away:.2f}")

    # --- VALUE FINDER ---
    bets_for_match = []
    
    for outcome, p, odds, label in [
        ('HOME', p_home, f['Odds_1'], h_team),
        ('DRAW', p_draw, f['Odds_X'], 'Draw'),
        ('AWAY', p_away, f['Odds_2'], a_team)
    ]:
        if odds > MAX_ODDS_CAP: continue
        
        # Note: We removed the "Goliath Filter" because the Market Probability Feature 
        # naturally handles this. If City is 1.20 odds, 'market_prob_away' will be high, 
        # forcing the model to predict City unless stats are INSANELY bad.
            
        edge = (p * odds) - 1
        min_p = 0.23 if outcome == 'DRAW' else 0.20
        
        if edge > MIN_EDGE and p > min_p:
            bets_for_match.append({'Type': outcome, 'Team': label, 'Odds': odds, 'Prob': p, 'Edge': edge})

    # Selection
    if bets_for_match:
        best = sorted(bets_for_match, key=lambda x: x['Edge'], reverse=True)[0]
        
        # Deadlock Logic (Preserved because it's still useful)
        gap = abs(p_home - p_away)
        if gap < 0.10:
            draw_bet = next((b for b in bets_for_match if b['Type'] == 'DRAW'), None)
            if draw_bet and best['Edge'] < (draw_bet['Edge'] + 0.15):
                best = draw_bet
                best['Reason'] = "Safe Draw"
        
        best['Match'] = f"{h_team} vs {a_team}"
        opportunities.append(best)

# ==============================================================================
# 4. PORTFOLIO
# ==============================================================================
if opportunities:
    res = pd.DataFrame(opportunities)
    
    def calculate_stake(row):
        b = row['Odds'] - 1
        p = row['Prob']
        q = 1 - p
        f = ((b * p) - q) / b
        return max(0, min(f * KELLY_FRACTION, MAX_STAKE_PCT))

    res['Stake_Pct'] = res.apply(calculate_stake, axis=1)
    res['Stake'] = res['Stake_Pct'] * BANKROLL
    res['Profit'] = res['Stake'] * (res['Odds'] - 1)
    
    res = res[res['Stake'] > 0.01].sort_values('Stake', ascending=False)
    
    display_df = res[['Match', 'Type', 'Team', 'Odds', 'Prob', 'Edge', 'Stake', 'Profit']].copy()
    display_df.columns = ['Match', 'Bet Type', 'Selection', 'Odds', 'Win %', 'Edge', 'Stake ($)', 'Profit ($)']
    
    styled = (display_df.style
        .format({'Odds': '{:.2f}', 'Win %': '{:.1%}', 'Edge': '{:.1%}', 'Stake ($)': '${:.2f}', 'Profit ($)': '${:.2f}'})
        .background_gradient(subset=['Win %'], cmap='Greens', vmin=0.2, vmax=0.8)
        .background_gradient(subset=['Edge'], cmap='Blues', vmin=0.0, vmax=0.5)
        .bar(subset=['Stake ($)'], color='#ff6b6b', vmin=0)
        .bar(subset=['Profit ($)'], color='#51cf66', vmin=0)
        .set_caption(f"🚀 MARKET-AWARE SLIP v19 | Bankroll: ${BANKROLL}")
    )
    display(styled)
    
    print(f"\n💰 Total Risk: ${res['Stake'].sum():.2f}")
else:
    print("\n📉 No Value Bets Found.")

   🎯 SNIPER ENGINE v2 (PRECISION + LOGIC DRAWS)       
✅ Sniper Models Loaded.

🔍 SCANNING TARGETS...
Manchester vs Newcastle : HomeConf 0.33 | AwayConf 0.39
Nottingham vs Manchester: HomeConf 0.09 | AwayConf 0.61
Arsenal vs Brighton &: HomeConf 0.83 | AwayConf 0.05
Brentford vs Bournemout: HomeConf 0.41 | AwayConf 0.24
Burnley vs Everton: HomeConf 0.20 | AwayConf 0.60
Liverpool vs Wolverhamp: HomeConf 0.90 | AwayConf 0.06
West Ham U vs Fulham: HomeConf 0.33 | AwayConf 0.47
Chelsea vs Aston Vill: HomeConf 0.41 | AwayConf 0.26
Sunderland vs Leeds Unit: HomeConf 0.34 | AwayConf 0.39
Crystal Pa vs Tottenham : HomeConf 0.50 | AwayConf 0.22


,Match,Bet Type,Selection,Odds,Win %,Edge,Signal,Stake ($),Profit ($)
4,Liverpool vs Wolverhampton Wanderers,HOME,Liverpool,1.24,89.9%,11.5%,Sniper Signal,$4.32,$1.04
1,Arsenal vs Brighton & Hove Albion,HOME,Arsenal,1.40,82.5%,15.5%,Sniper Signal,$3.49,$1.40
3,Burnley vs Everton,AWAY,Everton,2.01,59.9%,20.4%,Sniper Signal,$1.82,$1.84
0,Manchester United vs Newcastle United,DRAW,Draw,3.61,29.0%,4.7%,Deadlock Logic,$0.08,$0.21
2,Brentford vs Bournemouth,DRAW,Draw,3.60,29.0%,4.4%,Deadlock Logic,$0.08,$0.20



💰 Total Risk: $9.79
